This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

4UwyPxbzwzibLKEDp4Gt


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json'
params = {'start_date' : '2020-12-01', 'end_date' : '2020-12-01', 'api_key' : API_KEY}
r_day = requests.get(url, params=params)
one_day = json.loads(r_day.text)
r_day.ok

True

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json.dumps(one_day, indent = 4))

{
    "dataset": {
        "id": 10095370,
        "dataset_code": "AFX_X",
        "database_code": "FSE",
        "name": "Carl Zeiss Meditec (AFX_X)",
        "description": "Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704",
        "refreshed_at": "2020-12-01T14:48:09.907Z",
        "newest_available_date": "2020-12-01",
        "oldest_available_date": "2000-06-07",
        "column_names": [
            "Date",
            "Open",
            "High",
            "Low",
            "Close",
            "Change",
            "Traded Volume",
            "Turnover",
            "Last Price of the Day",
            "Daily Traded Units",
            "Daily Turnover"
        ],
        "frequency": "daily",
        "type": "Time Series",
        "premium": false,
        "limit": null,
        "transform": null,
        "column_index": null,
        "start_date": "2020-12-01",
        "end_date": 

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# tasks 1 and 2
params = {'start_date' : '2017-01-01', 'end_date' : '2017-12-31', 'api_key' : API_KEY}
r_2017 = requests.get(url, params=params)
r_2017.ok
all_2017 = json.loads(r_2017.text)["dataset"]
#print(all_2017["column_names"])

In [6]:
# task 3 - opening prices
high = 0
low = float('inf')
#print(json.dumps(all_2017["data"], indent=2))
for day in all_2017["data"]:
    if not day[1]:
        continue
    if day[1] > high:
        high = day[1]
    if day[1] < low:
        low = day[1]
print("high: $" + str(high))
print("low: $" + str(low))

high: $53.11
low: $34.0


In [7]:
# task 4 - daily high and low prices
max_day_change = 0
for day in all_2017["data"]:
    #if not day[2] or not day[3]:
    #    continue
    day_change = day[2] - day[3]
    max_day_change = max(max_day_change, day_change)
print("max daily change: $" + str(max_day_change))

max daily change: $2.8100000000000023


In [8]:
# task 5 - closing prices --> congruent days
max_closing_change = 0
prev_close = all_2017["data"][0][4]
for d in all_2017["data"]:
    curr_closing_change = abs(d[4] - prev_close)
    max_closing_change = max(max_closing_change, curr_closing_change)
print("greatest change in close between two congruent days: " + str(max_closing_change))

greatest change in close between two congruent days: 17.699999999999996


In [9]:
# task 5 - closing prices --> any 2 days
max_close = 0
min_close = float('inf')
for d in all_2017["data"]:
    if not d[4]:
        continue
    max_close = max(max_close, d[4])
    min_close = min(min_close, d[4])
close_dif = max_close - min_close
print("greatest change in close between any two days: " + str(close_dif))

greatest change in close between any two days: 19.03


In [10]:
# task 6 - avg trading volume
year_volume = 0
for d in all_2017["data"]:
    year_volume += d[6]
avg_volume = year_volume / len(all_2017["data"])
print("daily volume avg: " + str(avg_volume))

daily volume avg: 89124.33725490196


In [11]:
# Task 7 - median volume
volumes = []
for d in all_2017["data"]:
    volumes.append(d[6])
volumes.sort()
median = volumes[len(volumes)//2]
print("median volume: " + str(median))

median volume: 76286.0
